In [28]:
import os
import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen, ImportExampleGen, StatisticsGen, SchemaGen, ExampleValidator
from tfx.utils.dsl_utils import external_input
from tfx.proto import example_gen_pb2

In [2]:
context = InteractiveContext()

In [12]:
base_dir = os.getcwd()
data_dir = os.path.join(os.pardir, "data")

output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
        example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2)
    ]))

In [20]:
examples = external_input(os.path.join(base_dir, data_dir, "csv"))
example_gen = CsvExampleGen(input=examples, output_config=output)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 7
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 7
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-09-26T13_55_06.659716-_6am3zu8/CsvExampleGen/examples/7"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:78956236,xor_checksum:1632657751,sum_checksum:1632657751"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "examples"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "pipeline_name"
          value {
            string_value: "interactive-2021-09-26T13_55_06.659716"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "CsvExampleGen"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

In [23]:
statistics_gen = StatisticsGen(examples=example_gen.outputs["examples"])
context.run(statistics_gen)

2021-09-26 14:07:18.368202: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-26 14:07:18.368233: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-26 14:07:18.368261: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2021-09-26 14:07:18.387834: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 3401925000 Hz
2021-09-26 14:07:18.388105: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55cbc60f6920 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-26 14:07:18.388176: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 8
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 8
        type_id: 7
        uri: "/tmp/tfx-interactive-2021-09-26T13_55_06.659716-_6am3zu8/StatisticsGen/statistics/8"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "pipeline_name"
          value {
            string_value: "interactive-2021-09-26T13_55_06.659716"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [25]:
context.show(statistics_gen.outputs['statistics'])

In [27]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)
context.run(schema_gen)

2021-09-26 14:10:25.330884: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:486] No property is defined for the Type


ExecutionResult(
    component_id: SchemaGen
    execution_id: 9
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 9
        type_id: 9
        uri: "/tmp/tfx-interactive-2021-09-26T13_55_06.659716-_6am3zu8/SchemaGen/schema/9"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "pipeline_name"
          value {
            string_value: "interactive-2021-09-26T13_55_06.659716"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

In [29]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'],
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 10
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 10
        type_id: 11
        uri: "/tmp/tfx-interactive-2021-09-26T13_55_06.659716-_6am3zu8/ExampleValidator/anomalies/10"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "pipeline_name"
          value {
            string_value: "interactive-2021-09-26T13_55_06.659716"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))